# 文本相似度

### Step1 导包

In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

### Step2 加载数据集

In [4]:
dataset = load_dataset("json", data_files="../data/train_pair_1w.json", split="train")
dataset

Dataset({
    features: ['sentence1', 'sentence2', 'label'],
    num_rows: 10000
})

In [6]:
dataset[0]

{'sentence1': '找一部小时候的动画片', 'sentence2': '求一部小时候的动画片。谢了', 'label': '1'}

In [8]:
datasets = dataset.train_test_split(test_size=0.2)
datasets

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label'],
        num_rows: 8000
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label'],
        num_rows: 2000
    })
})

### Step3 数据预处理

In [10]:
import torch

tokenizer = AutoTokenizer.from_pretrained("../hfl/chinese-macbert-base")

def process_function(examples):
    tokenized_examples = tokenizer(examples["sentence1"], examples["sentence2"], max_length=128, truncation=True)
    tokenized_examples["labels"] = [float(label) for label in examples["label"]]
    return tokenized_examples

tokenized_datasets = datasets.map(process_function, batched=True, remove_columns=datasets["train"].column_names)
tokenized_datasets

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 8000
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 2000
    })
})

### Step4 创建模型

In [16]:
# from transformers import BertForSequenceClassification # 可以进入BertForSequenceClassification这个类查看num_labels=1时其处理方式（主要原理是当num_labels=1时其将这个任务视为回归任务）

model = AutoModelForSequenceClassification.from_pretrained("../hfl/chinese-macbert-base", num_labels=1)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ../hfl/chinese-macbert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Step5 创建评估函数

In [17]:
import evaluate

acc_metric = evaluate.load("metric_accuracy.py")
f1_metric = evaluate.load("metric_f1.py")

In [23]:
def eval_metric(eval_predict):
    predictions, labels = eval_predict
    predictions = [int(p > 0.5) for p in predictions]
    labels = [int(l) for l in labels]
    acc = acc_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels)
    acc.update(f1)
    return acc

### Step6 TrainingArguments

In [24]:
train_args = TrainingArguments(
    output_dir="v1_result",
    per_device_eval_batch_size=32,
    per_device_train_batch_size=32,
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=3,
    learning_rate=2e-5,
    weight_decay=0.01,
    metric_for_best_model="f1",
    load_best_model_at_end=True
)

d:\miniconda3\envs\llm\lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


### Step7 Trainer

In [25]:
from transformers import DataCollatorWithPadding
trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=train_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=eval_metric
)

C:\Users\柒\AppData\Local\Temp\ipykernel_33980\1995911572.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


### Step8 模型训练

In [26]:
trainer.train()

  0%|          | 0/750 [00:00<?, ?it/s]

{'loss': 0.094, 'grad_norm': 6.957308292388916, 'learning_rate': 1.9733333333333336e-05, 'epoch': 0.04}
{'loss': 0.0689, 'grad_norm': 2.536043643951416, 'learning_rate': 1.9466666666666668e-05, 'epoch': 0.08}
{'loss': 0.0621, 'grad_norm': 1.3592231273651123, 'learning_rate': 1.9200000000000003e-05, 'epoch': 0.12}
{'loss': 0.0587, 'grad_norm': 4.83639669418335, 'learning_rate': 1.8933333333333334e-05, 'epoch': 0.16}
{'loss': 0.079, 'grad_norm': 4.822652816772461, 'learning_rate': 1.866666666666667e-05, 'epoch': 0.2}
{'loss': 0.0563, 'grad_norm': 3.1282684803009033, 'learning_rate': 1.8400000000000003e-05, 'epoch': 0.24}
{'loss': 0.0481, 'grad_norm': 3.0717780590057373, 'learning_rate': 1.8133333333333335e-05, 'epoch': 0.28}
{'loss': 0.044, 'grad_norm': 2.2744767665863037, 'learning_rate': 1.7866666666666666e-05, 'epoch': 0.32}
{'loss': 0.046, 'grad_norm': 2.238105058670044, 'learning_rate': 1.76e-05, 'epoch': 0.36}
{'loss': 0.0549, 'grad_norm': 2.345521926879883, 'learning_rate': 1.7333

  0%|          | 0/63 [00:00<?, ?it/s]

{'eval_loss': 0.0736842006444931, 'eval_accuracy': 0.9065, 'eval_f1': 0.8714776632302406, 'eval_runtime': 6.6633, 'eval_samples_per_second': 300.151, 'eval_steps_per_second': 9.455, 'epoch': 1.0}
{'loss': 0.0554, 'grad_norm': 7.634251117706299, 'learning_rate': 1.3066666666666668e-05, 'epoch': 1.04}
{'loss': 0.0682, 'grad_norm': 5.4067487716674805, 'learning_rate': 1.2800000000000001e-05, 'epoch': 1.08}
{'loss': 0.0689, 'grad_norm': 5.286261081695557, 'learning_rate': 1.2533333333333336e-05, 'epoch': 1.12}
{'loss': 0.0679, 'grad_norm': 3.5509519577026367, 'learning_rate': 1.2266666666666667e-05, 'epoch': 1.16}
{'loss': 0.0454, 'grad_norm': 1.7824488878250122, 'learning_rate': 1.2e-05, 'epoch': 1.2}
{'loss': 0.0489, 'grad_norm': 1.3300397396087646, 'learning_rate': 1.1733333333333335e-05, 'epoch': 1.24}
{'loss': 0.0674, 'grad_norm': 10.062271118164062, 'learning_rate': 1.1466666666666668e-05, 'epoch': 1.28}
{'loss': 0.0542, 'grad_norm': 2.6538259983062744, 'learning_rate': 1.12000000000

  0%|          | 0/63 [00:00<?, ?it/s]

{'eval_loss': 0.06763548403978348, 'eval_accuracy': 0.917, 'eval_f1': 0.8909329829172141, 'eval_runtime': 6.5248, 'eval_samples_per_second': 306.521, 'eval_steps_per_second': 9.655, 'epoch': 2.0}
{'loss': 0.0315, 'grad_norm': 1.7196966409683228, 'learning_rate': 6.4000000000000006e-06, 'epoch': 2.04}
{'loss': 0.0429, 'grad_norm': 1.5235246419906616, 'learning_rate': 6.133333333333334e-06, 'epoch': 2.08}
{'loss': 0.0509, 'grad_norm': 2.693058490753174, 'learning_rate': 5.8666666666666675e-06, 'epoch': 2.12}
{'loss': 0.0409, 'grad_norm': 4.674600601196289, 'learning_rate': 5.600000000000001e-06, 'epoch': 2.16}
{'loss': 0.0502, 'grad_norm': 2.0740509033203125, 'learning_rate': 5.333333333333334e-06, 'epoch': 2.2}
{'loss': 0.0356, 'grad_norm': 2.2640721797943115, 'learning_rate': 5.0666666666666676e-06, 'epoch': 2.24}
{'loss': 0.0413, 'grad_norm': 2.2498908042907715, 'learning_rate': 4.800000000000001e-06, 'epoch': 2.28}
{'loss': 0.0386, 'grad_norm': 1.6661852598190308, 'learning_rate': 4.

  0%|          | 0/63 [00:00<?, ?it/s]

{'eval_loss': 0.06780592352151871, 'eval_accuracy': 0.9165, 'eval_f1': 0.8900592495062543, 'eval_runtime': 6.4317, 'eval_samples_per_second': 310.96, 'eval_steps_per_second': 9.795, 'epoch': 3.0}
{'train_runtime': 291.1668, 'train_samples_per_second': 82.427, 'train_steps_per_second': 2.576, 'train_loss': 0.051644726713498436, 'epoch': 3.0}


TrainOutput(global_step=750, training_loss=0.051644726713498436, metrics={'train_runtime': 291.1668, 'train_samples_per_second': 82.427, 'train_steps_per_second': 2.576, 'total_flos': 1556485250567424.0, 'train_loss': 0.051644726713498436, 'epoch': 3.0})

### Step9 模型评估

In [27]:
trainer.evaluate(tokenized_datasets["test"])

  0%|          | 0/63 [00:00<?, ?it/s]

{'eval_loss': 0.06763548403978348,
 'eval_accuracy': 0.917,
 'eval_f1': 0.8909329829172141,
 'eval_runtime': 6.6259,
 'eval_samples_per_second': 301.845,
 'eval_steps_per_second': 9.508,
 'epoch': 3.0}

### Step10 模型预测

In [28]:
from transformers import pipeline, TextClassificationPipeline

In [29]:
model.config.id2label = {0: "不相似", 1: "相似"}

In [30]:
pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0)

In [31]:
result = pipe({"text": "我喜欢北京", "text_pair":"天气怎样"}, function_to_apply="none")
result["label"] = "相似" if result["score"] > 0.5 else "不相似"
result

{'label': '不相似', 'score': 0.03655894473195076}